### **4 - Especificando el parser a utilizar**

Si sólo necesitas parsear algo de HTML, puedes "dump the markup" en el constructor BeautifulSoup, y probablemente estará bien. Beautiful Soup elegirá un parser por ti y parseará los datos. Pero hay algunos argumentos adicionales que puedes pasar al constructor para cambiar el parser utilizado.

El primer argumento del constructor de BeautifulSoup es un string o un filehandle abierto - el markup que quieres parsear. El segundo argumento es cómo quieres que se parsee el markup.

Si no especificas nada, obtendrás el mejor parseador HTML que esté instalado. Beautiful Soup clasifica el parser de lxml como el mejor, luego el de html5lib, luego el parser integrado de Python. Puedes anular esto especificando uno de los siguientes:

- Qué tipo de markup quieres parsear. Actualmente se soportan "html", "xml" y "html5".
- El nombre de la librería de parseo que desea utilizar. Las opciones actualmente soportadas son "lxml", "html5lib" y "html.parser" (el analizador HTML integrado en Python).

Si no tienes instalado un parser apropiado, Beautiful Soup ignorará tu re questy elegirá un parser diferente. En este momento, el único parser XML soportado es lxml. Si no tienes lxml instalado, pedir un parser XML no te dará uno, y pedir "lxml" tampoco funcionará.

#### **Diferencias entre parsers**

Beautiful Soup presenta la misma interfaz a diferentes parsers, pero cada parser es diferente. Diferentes parsers crearán diferentes árboles de parseo a partir del mismo documento. Las mayores diferencias están entre los analizadores HTML y los analizadores XML. Aquí tienes un documento corto, analizado como HTML:

In [1]:
from bs4 import BeautifulSoup

soup = BeautifulSoup("<a><b /></a>")
print(soup.prettify())

<html>
 <body>
  <a>
   <b>
   </b>
  </a>
 </body>
</html>



Dado que una etiqueta `<b />` vacía no es HTML válido, el parser la convierte en un par de etiquetas `<b></b>`.

Aquí está el mismo documento analizado como XML (para ejecutarlo es necesario tener `lxml` instalado). Tenga en cuenta que la etiqueta `<b />` vacía se deja sola y que el documento recibe una declaración XML en lugar de colocarse en una etiqueta `<html>`.:

In [4]:
soup = BeautifulSoup("<a><b /></a>", "xml")
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<a>
 <b/>
</a>



También hay diferencias entre los parsers HTML. Si le das a Beautiful Soup un documento HTML perfectamente formado, estas diferencias no importarán. Un parser será más rápido que otro, pero todos te darán una estructura de datos exactamente igual a la del documento HTML original.

Pero si el documento no está perfectamente formado, los distintos parseadores darán resultados diferentes. He aquí un documento corto no válido analizado con el parser HTML de lxml. Observe que la etiqueta `</p>` colgante simplemente se ignora:

In [5]:
soup = BeautifulSoup("<a></p>", "lxml")
print(soup.prettify())

<html>
 <body>
  <a>
  </a>
 </body>
</html>



Aquí está el mismo documento parseado usando `html5lib`:

In [7]:
pip install html5lib

  Obtaining dependency information for html5lib from https://files.pythonhosted.org/packages/6c/dd/a834df6482147d48e225a49515aabc28974ad5a4ca3215c18a882565b028/html5lib-1.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for webencodings from https://files.pythonhosted.org/packages/f4/24/2a3e3df732393fed8b3ebf2ec078f05546de641fe1b667ee316ec1dcf3b7/webencodings-0.5.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/112.2 kB ? eta -:--:--
   --- ------------------------------------ 10.2/112.2 kB ? eta -:--:--
   ------------------------ -------------- 71.7/112.2 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 112.2/112.2 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
soup = BeautifulSoup("<a></p>", "html5lib")
print(soup.prettify())

<html>
 <head>
 </head>
 <body>
  <a>
   <p>
   </p>
  </a>
 </body>
</html>



En lugar de ignorar la etiqueta `</p>` colgante, html5lib la empareja con una etiqueta `<p>` de apertura. Este parser también añade una etiqueta `<head>` vacía al documento.

Aquí está el mismo documento analizado con el parser HTML integrado de Python:

In [3]:
soup = BeautifulSoup("<a></p>", "html.parser")
print(soup.prettify())

<a>
</a>



Al igual que html5lib, este parser ignora la etiqueta de cierre `</p>`. A diferencia de html5lib, este parser no intenta crear un documento HTML bien formado añadiendo una etiqueta `<body>`. A diferencia de lxml, ni siquiera se molesta en añadir una etiqueta `<html>`.

Dado que el documento "`<a></p>`" no es válido, ninguna de estas técnicas es la forma "correcta" de manejarlo. El parser html5lib utiliza técnicas que son parte del estándar HTML5, por lo que tiene la mejor reclamación de ser la forma "correcta", pero las tres técnicas son legítimas.

Las diferencias entre los parsers pueden afectar a tu script. Si estás planeando distribuir tu script a otras personas, o ejecutarlo en múltiples máquinas, deberías especificar un parser en el constructor de BeautifulSoup. Eso reducirá las posibilidades de que tus usuarios parseen un documento de forma diferente a como tú lo parseas.